In [1]:
# FIX BUG WITH SQLITE
# https://gist.github.com/defulmere/8b9695e415a44271061cc8e272f3c300

__import__('pysqlite3')
import sys
import os
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.sqlite3',
        'NAME': os.path.join('..', 'db.sqlite3'),
    }
}

In [2]:
import os
import openai
from dotenv import load_dotenv
from langchain_community.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain_community.llms import OpenAI
from langchain.chains import VectorDBQA
from langchain.schema.document import Document

In [3]:
from langchain_community.document_loaders.csv_loader import CSVLoader

In [4]:
loader = CSVLoader(file_path=os.path.join(sys.path[0], "../src/artifacts/@karpovcourseschat.csv"), source_column="text", metadata_columns=["date", "message_id"], csv_args={
    "delimiter": ";",
})
data = loader.load()

In [5]:
print(data)

[Document(page_content='text: а я сильно загружал: обучался питону. ролики, сайты, редактор кода. одних вкладок открыто постоянно 50-100', metadata={'source': 'а я сильно загружал: обучался питону. ролики, сайты, редактор кода. одних вкладок открыто постоянно 50-100', 'row': 0, 'date': '2024-01-05 11:11:44+00:00', 'message_id': '192405'}), Document(page_content='text: Не особо', metadata={'source': 'Не особо', 'row': 1, 'date': '2024-01-05 11:05:58+00:00', 'message_id': '192404'}), Document(page_content='text: видимо, нормально для этих моделей 🤔', metadata={'source': 'видимо, нормально для этих моделей 🤔', 'row': 2, 'date': '2024-01-05 11:02:47+00:00', 'message_id': '192403'}), Document(page_content='text: оперативу загружаешь?', metadata={'source': 'оперативу загружаешь?', 'row': 3, 'date': '2024-01-05 11:02:25+00:00', 'message_id': '192402'}), Document(page_content='text: У меня такой мак уже год, 90% времени подключен к монитору, 98% аккум', metadata={'source': 'У меня такой мак уж

In [6]:
load_dotenv(dotenv_path='../.env')
PROXY_API_KEY = os.getenv("PROXY_API_KEY")

In [7]:
emb_fn = OpenAIEmbeddings(
    api_key=os.getenv("PROXY_API_KEY"),
    model="text-embedding-ada-002",
    base_url="https://api.proxyapi.ru/openai/v1",
)

/home/kartushov/pet_projects/PostFinder/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.OpenAIEmbeddings instead.
  warn_deprecated(


In [8]:
db = Chroma.from_documents(data, emb_fn)

In [9]:
query = "Ролики по питону"
docs = db.similarity_search(query)

In [10]:
print(docs)

[Document(page_content='text: а я сильно загружал: обучался питону. ролики, сайты, редактор кода. одних вкладок открыто постоянно 50-100', metadata={'date': '2024-01-05 11:11:44+00:00', 'message_id': '192405', 'row': 0, 'source': 'а я сильно загружал: обучался питону. ролики, сайты, редактор кода. одних вкладок открыто постоянно 50-100'}), Document(page_content='text: видимо, нормально для этих моделей 🤔', metadata={'date': '2024-01-05 11:02:47+00:00', 'message_id': '192403', 'row': 2, 'source': 'видимо, нормально для этих моделей 🤔'}), Document(page_content='text: оперативу загружаешь?', metadata={'date': '2024-01-05 11:02:25+00:00', 'message_id': '192402', 'row': 3, 'source': 'оперативу загружаешь?'}), Document(page_content='text: бу сколько', metadata={'date': '2024-01-05 10:58:27+00:00', 'message_id': '192396', 'row': 9, 'source': 'бу сколько'})]


In [11]:
import httpx

openai = OpenAI(openai_api_key=PROXY_API_KEY, openai_api_base="https://api.proxyapi.ru/openai/v1")

/home/kartushov/pet_projects/PostFinder/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.OpenAI instead.
  warn_deprecated(


In [14]:
from langchain.chat_models import ChatOpenAI

In [15]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=PROXY_API_KEY, openai_api_base="https://api.proxyapi.ru/openai/v1")

/home/kartushov/pet_projects/PostFinder/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


In [16]:
qa = VectorDBQA.from_chain_type(llm=llm, chain_type="stuff", vectorstore=db)

/home/kartushov/pet_projects/PostFinder/.venv/lib/python3.10/site-packages/langchain/chains/retrieval_qa/base.py:256: UserWarning: `VectorDBQA` is deprecated - please use `from langchain.chains import RetrievalQA`
  warnings.warn(


In [17]:
qa.run(query)

'Я не знаю, сколько роликов по питону было просмотрено.'